In [10]:
import os

In [11]:
%pwd

'd:\\Project\\P\\Mlops-Project'

In [3]:
os.chdir("../")
%pwd

'd:\\Project\\P\\Mlops-Project'

In [28]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    dataset_path: Path
    schema: list
    params: dict
    mlflow_uri: str
    

In [19]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/MorretiKlein/Mlops-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="MorretiKlein"
os.environ["MLFLOW_TRACKING_PASSWORD"]="c1c64d907121622a72df895d266f6d75a86c8d50"

In [20]:
from MLOps_project.constant import *
from MLOps_project import logger
from MLOps_project.utils.common import read_yaml, create_directories, save_json

In [21]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.yolov5
        schema =  self.schema.COLUMNS

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            dataset_path = config.dataset_path
            schema = [schema.XMin, schema.XMax, schema.YMin, schema.YMax],
            params = params ,
            mlflow_uri="https://dagshub.com/MorretiKlein/Mlops-Project",
           
        )

        return model_evaluation_config

In [22]:
import mlflow
from urllib.parse import urlparse

In [26]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme # get registry of MLflow

        if tracking_url_type_store != "file":
            logger.info("Logging into MLflow...")
            with mlflow.start_run():
                mlflow.log_params(self.config.params)
                mlflow.log_artifacts(self.config.root_dir)
        else:
            logger.warning("MLflow is configured to store data in local files.")

In [27]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2023-12-21 15:38:41,585: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-21 15:38:41,585: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-21 15:38:41,585: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-21 15:38:41,601: INFO: common: created directory at: artifacts]
[2023-12-21 15:38:41,604: INFO: common: created directory at: evaluation/]
[2023-12-21 15:38:41,608: INFO: 3885834704: Logging into MLflow...]


[2023-12-21 15:39:00,921: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /MorretiKlein/Mlops-Project.mlflow/api/2.0/mlflow/runs/create]


In [ ]:
c1c64d907121622a72df895d266f6d75a86c8d50 